# Telangana Crop Health Challenge
---
###  Can you predict the health status of crops?




---



# Libraries and Modules Import

In [1]:
# Standard Library Imports
from concurrent.futures import ThreadPoolExecutor, as_completed
from joblib import Parallel, delayed
from tqdm import tqdm
from datetime import datetime
from pathlib import Path
import os
import random

# Third-Party Imports
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from lightgbm import LGBMClassifier
from shapely.affinity import scale, translate
from skimage import exposure
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder

from shapely import wkt
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

import logging
# Set up a logger to capture Rasterio warnings
logging.getLogger("rasterio._env").setLevel(logging.ERROR)

# Project Setup and Earth Engine Initialization

In [2]:
# Define the root path for the project
root_path = Path(".")
root_path

WindowsPath('.')

In [3]:
# Authenticate with Google Earth Engine
# This opens a browser prompt for authentication, if not previously authenticated
ee.Authenticate()

# Initialize Earth Engine with a specific project
# Replace "project" with your project ID as needed
ee.Initialize(project="zinditelangana2024")

# Data Loading and Preprocessing

**Load Train and Test**

In [4]:
# Load training and testing datasets from CSV files
train = pd.read_csv(root_path / 'train.csv')
test = pd.read_csv(root_path / 'test.csv')

**Convert Geometry Columns to Geospatial Format**

In [5]:
# Convert WKT geometry to actual geometry objects in both datasets
train['geometry'] = train['geometry'].apply(wkt.loads)
test['geometry'] = test['geometry'].apply(wkt.loads)

**Convert DataFrames to GeoDataFrames**

In [6]:
# Convert pandas DataFrames to GeoDataFrames with CRS set to 'epsg:4326'
train = gpd.GeoDataFrame(train, crs='epsg:4326')
test = gpd.GeoDataFrame(test, crs='epsg:4326')

**Concatenate Train and Test Data for Unified Processing**

In [7]:
# Concatenate train and test datasets into a single DataFrame for consistent processing
# 'dataset' column distinguishes between train and test rows
data = pd.concat(
    [train.assign(dataset='train'), test.assign(dataset='test')]
).reset_index(drop=True)

**Scale Geometry Objects**

In [8]:
# Function to scale each geometry around its centroid
def scale_geometry(geometry, scale_factor):
    """
    Scale a geometry object around its centroid.

    Parameters:
        geometry (shapely.geometry.base.BaseGeometry): Geometry to scale.
        scale_factor (float): Factor by which to scale the geometry.

    Returns:
        shapely.geometry.base.BaseGeometry: Scaled geometry object.
    """
    # Calculate centroid
    centroid = geometry.centroid
    # Translate geometry to origin
    translated_geometry = translate(geometry, -centroid.x, -centroid.y)
    # Scale geometry
    scaled_geometry = scale(translated_geometry, xfact=scale_factor, yfact=scale_factor, origin=(0, 0))
    # Translate back to the original centroid
    return translate(scaled_geometry, centroid.x, centroid.y)

In [9]:
# Set the scaling factor
scale_factor = 5

# Apply the scaling function to each geometry in the data
# This modifies each geometry in place by the defined scale factor
data['geometry'] = data['geometry'].apply(scale_geometry, scale_factor=scale_factor)

# Download Image Data

**Utils**

In [10]:
# Function to ensure date format is correct
def format_date(date_str):
    """Convert date string to 'YYYY-MM-DD' format."""
    try:
        return datetime.strptime(date_str, '%d-%m-%Y').strftime('%Y-%m-%d')
    except ValueError:
        print(f"Error: Invalid date format for {date_str}. Expected 'DD-MM-YYYY'.")
        return None

**Convert GeoDataFrame to Earth Engine FeatureCollection**

In [11]:
# Function to convert a GeoDataFrame to Google Earth Engine FeatureCollection
def gdf_to_gee(gdf):
    """Converts a GeoDataFrame to an Earth Engine FeatureCollection."""
    features = []
    for _, row in gdf.iterrows():
        geom = row['geometry'].__geo_interface__  # Convert geometry to GeoJSON format
        feature = ee.Feature(ee.Geometry(geom), row.to_dict())  # Create an EE Feature
        features.append(feature)
    return ee.FeatureCollection(features)

# Convert the entire dataset GeoDataFrame to a FeatureCollection for GEE
gee_feature_collection = gdf_to_gee(data)

**Initialize Earth Engine and Set Up Directories**

In [12]:
# if you already downloaded the data.
donwload = False

# Initialize Earth Engine
ee.Initialize()

# Remove existing 'downloads' folder if it exists
if os.path.exists('downloads'):
    os.system('rm -rf downloads')

# Ensure the output directory exists
output_folder = 'downloads'
os.makedirs(output_folder, exist_ok=True)

**Download from Earth Engine**

In [13]:
# Function to download RGB images from Earth Engine
def download_rgb_image(collection_name, bands, start_date, end_date, region, output_folder='downloads'):
    """Download RGB bands from a GEE collection filtered by date and region."""
    # Load the image collection, filter by date, and clip to region
    collection = ee.ImageCollection(collection_name).filterDate(start_date, end_date).filterBounds(region)
    image = collection.sort('system:time_start', False).first().select(bands).clip(region)  # Most recent image

    # Define unique filename based on image dates
    image_id = image.id().getInfo() or f'image_{start_date}_{end_date}'
    image_name = f'{output_folder}/{image_id}_RGB_{start_date}_{end_date}.tif'

    # Export the image to a GeoTIFF file
    geemap.ee_export_image(
        image,
        filename=image_name,
        scale=10,  # Sentinel-2 resolution in meters
        region=region,
        file_per_band=False,  # Save as a multi-band TIFF
        crs='EPSG:4326'
    )
    print(f"Downloaded: {image_name}")
    return image_name

In [14]:
# Function to process each row in the dataset for image download
def process_row(index, row):
    # Format start and end dates
    start_date = format_date(row['SDate'])
    end_date = format_date(row['HDate'])

    # Skip rows with invalid dates
    if not start_date or not end_date:
        print(f"Skipping entry due to invalid dates: SDate={row['SDate']}, HDate={row['HDate']}")
        return index, None

    # Extract and check geometry
    region_geometry = row['geometry']
    district = row['District']

    # Verify valid geometry and convert it to EE format
    if region_geometry.is_empty:
        print(f"Skipping entry due to empty geometry for District: {district}")
        return index, None

    # Convert geometry type to EE compatible format
    if region_geometry.geom_type == 'Polygon':
        region = ee.Geometry.Polygon(region_geometry.__geo_interface__['coordinates'])
    elif region_geometry.geom_type == 'MultiPolygon':
        coords = [polygon.exterior.coords[:] for polygon in region_geometry.geoms]
        region = ee.Geometry.MultiPolygon(coords)
    else:
        print(f"Skipping unsupported geometry type: {region_geometry.geom_type} for District: {district}")
        return index, None

    # Define Sentinel-2 collection and bands
    image_collection_name = 'COPERNICUS/S2'
    bands = ['B4', 'B3', 'B2', 'B8', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12']

    # Attempt to download the image
    try:
        image_file = download_rgb_image(image_collection_name, bands, start_date, end_date, region, output_folder)
        return index, image_file
    except Exception as e:
        print(f"Error processing entry for District {district}: {e}")
        return index, None

In [15]:
import pandas as pd

# Assuming 'SDate' and 'HDate' are the columns with dates
data['SDate'] = pd.to_datetime(data['SDate'], errors='coerce').dt.strftime('%d-%m-%Y')
data['HDate'] = pd.to_datetime(data['HDate'], errors='coerce').dt.strftime('%d-%m-%Y')

# Check for rows with invalid dates after conversion
invalid_rows = data[data['SDate'].isnull() | data['HDate'].isnull()]
print("Rows with invalid dates:")
print(invalid_rows)
donwload = True

Rows with invalid dates:
Empty GeoDataFrame
Columns: [FarmID, category, Crop, State, District, Sub-District, SDate, HDate, CropCoveredArea, CHeight, CNext, CLast, CTransp, IrriType, IrriSource, IrriCount, WaterCov, ExpYield, Season, geometry, dataset]
Index: []

[0 rows x 21 columns]


In [21]:
if donwload:
  # Execute image downloads for each row in parallel to improve performance
  with ThreadPoolExecutor(max_workers=50) as executor:  # Adjust the number of workers as needed
      futures = [executor.submit(process_row, index, row) for index, row in data.iterrows()]
      for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading Images", unit="image"): 
          index, image_file = future.result()
          if image_file:
              data.at[index, 'tif_path'] = image_file  # Store the downloaded file path
else:
  # Load existing data with image paths if downloads are not required
  os.system('cp -r ./downloads .')
  data_path = "./data.csv"

  data = pd.read_csv(data_path)

c:\Users\chavv\anaconda\envs\telangana\lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)

Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...


Data downloaded to d:\github\telangana-crop-health\downloads\20240412T050649_20240412T051727_T43QHV_RGB_2023-11-25_2024-04-14.tif
Downloaded: downloads/20240412T050649_20240412T051727_T43QHV_RGB_2023-11-25_2024-04-14.tif
Data downloaded to d:\github\telangana-crop-health\downloads\20240517T050651_20240517T051655_T43QHV_RGB_2023-12-13_2024-05-18.tif
Downloaded: downloads/20240517T050651_20240517T051655_T43QHV_RGB_2023-12-13_2024-05-18.tif
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Generating URL ...


Data downloaded to d:\github\telangana-crop-health\downloads\20240427T050651_20240427T051439_T43QHV_RGB_2023-12-19_2024-04-28.tif
Downloaded: downloads/20240427T050651_20240427T051439_T43QHV_RGB_2023-12-19_2024-04-28.tif
Data downloaded to d:\github\telangana-crop-health\downloads\20240517T050651_20240517T051655_T43QHV_RGB_2023-12-12_2024-05-19.tif
Downloaded: downloads/20240517T050651_20240517T051655_T43QHV_RGB_2023-12-12_2024-05-19.tif
Data downloaded to d:\github\telangana-crop-health\downloads\20241004T050651_20241004T052234_T43QHV_RGB_2023-10-12_2024-10-05.tif
Downloaded: downloads/20241004T050651_20241004T052234_T43QHV_RGB_2023-10-12_2024-10-05.tif
Please wait ...
Data downloaded to d:\github\telangana-crop-health\downloads\20240425T051649_20240425T052817_T43QHV_RGB_2023-11-13_2024-04-26.tif
Downloaded: downloads/20240425T051649_20240425T052817_T43QHV_RGB_2023-11-13_2024-04-26.tif
Data downloaded to d:\github\telangana-crop-health\downloads\20240512T050649_20240512T051901_T43QHV_

Data downloaded to d:\github\telangana-crop-health\downloads\20240527T050651_20240527T051452_T43QHV_RGB_2023-02-12_2024-05-29.tif
Downloaded: downloads/20240527T050651_20240527T051452_T43QHV_RGB_2023-02-12_2024-05-29.tif


...

Generating URL ...
Data downloaded to d:\github\telangana-crop-health\downloads\20240601T050649_20240601T051727_T43QHB_RGB_2023-06-10_2024-06-02.tif
Downloaded: downloads/20240601T050649_20240601T051727_T43QHB_RGB_2023-06-10_2024-06-02.tif
Please wait ...
Please wait ...
Data downloaded to d:\github\telangana-crop-health\downloads\20240402T050649_20240402T051718_T43QHB_RGB_2023-03-12_2024-04-03.tif
Downloaded: downloads/20240402T050649_20240402T051718_T43QHB_RGB_2023-03-12_2024-04-03.tif


Data downloaded to d:\github\telangana-crop-health\downloads\20240321T051651_20240321T053116_T43QHB_RGB_2023-11-11_2024-03-23.tif
Downloaded: downloads/20240321T051651_20240321T053116_T43QHB_RGB_2023-11-11_2024-03-23.tif
Data downloaded to d:\github\telangana-crop-health\downloads\20240318T050651_20240318T052150_T43QHB_RGB_2023-10-16_2024-03-20.tif
Downloaded: downloads/20240318T050651_20240318T052150_T43QHB_RGB_2023-10-16_2024-03-20.tif
Please wait ...
Please wait ...


Data downloaded to d:\github\telangana-crop-health\downloads\20241128T051059_20241128T051941_T43QHB_RGB_2023-08-11_2024-12-03.tif
Downloaded: downloads/20241128T051059_20241128T051941_T43QHB_RGB_2023-08-11_2024-12-03.tif
Generating URL ...


Generating URL ...
Please wait ...
Data downloaded to d:\github\telangana-crop-health\downloads\20240222T050829_20240222T051835_T43QHB_RGB_2023-02-10_2024-02-26.tif
Downloaded: downloads/20240222T050829_20240222T051835_T43QHB_RGB_2023-02-10_2024-02-26.tif
Generating URL ...
Generating URL ...
Generating URL ...


Generating URL ...
Data downloaded to d:\github\telangana-crop-health\downloads\20240217T050901_20240217T051716_T43QHB_RGB_2023-12-22_2024-02-18.tif
Downloaded: downloads/20240217T050901_20240217T051716_T43QHB_RGB_2023-12-22_2024-02-18.tif
Data downloaded to d:\github\telangana-crop-health\downloads\20240222T050829_20240222T051835_T43QHB_RGB_2023-12-29_2024-02-23.tif
Downloaded: downloads/20240222T050829_20240222T051835_T43QHB_RGB_2023-12-29_2024-02-23.tif
Data downloaded to d:\github\telangana-crop-health\downloads\20240326T051649_20240326T053300_T43QHB_RGB_2023-02-11_2024-03-27.tif
Downloaded: downloads/20240326T051649_20240326T053300_T43QHB_RGB_2023-02-11_2024-03-27.tif


# Visualization

In [22]:
data

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,...,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,dataset
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25 0:00:00,2024-04-14 0:00:00,97,54,...,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,"POLYGON ((78.18079 17.97897, 78.1811 17.97948,...",train
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13 0:00:00,2024-04-26 0:00:00,82,58,...,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,"POLYGON ((78.17483 17.98151, 78.17649 17.98136...",train
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19 0:00:00,2024-04-28 0:00:00,92,91,...,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,"POLYGON ((78.16888 17.97673, 78.16968 17.97661...",train
3,1331836,Diseased,Paddy,Telangana,Medak,Kulcharam,2023-02-11 0:00:00,2024-11-04 0:00:00,91,52,...,Lentil,Transplanting,Flood,Canal,5,92,16,Rabi,"POLYGON ((78.16822 17.97469, 78.16955 17.97522...",train
4,1326579,Diseased,Paddy,Telangana,Medak,Kulcharam,2023-12-12 0:00:00,2024-05-19 0:00:00,94,55,...,Lentil,Transplanting,Flood,Canal,5,97,20,Rabi,"POLYGON ((78.17244 17.96962, 78.17303 17.96964...",train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10601,551119,NaN,Maize,Telangana,Nirmal,Sarangapur,2023-10-12 0:00:00,2024-07-03 0:00:00,92,225,...,Maize,Drilling,Drip,Groundwater,4,44,39,Rabi,"POLYGON ((78.259 19.22927, 78.25973 19.22915, ...",test
10602,916476,NaN,Maize,Telangana,Nirmal,Sarangapur,2023-05-11 0:00:00,2024-04-03 0:00:00,79,236,...,Maize,Seed Drilling,Drip,Canal,3,41,40,Rabi,"POLYGON ((78.25819 19.20201, 78.25792 19.20033...",test
10603,121860,NaN,Maize,Telangana,Nirmal,Sarangapur,2023-11-14 0:00:00,2024-03-04 0:00:00,87,210,...,Maize,Broadcasting,Sprinkler,Canal,3,43,41,Rabi,"POLYGON ((78.25546 19.19637, 78.2556 19.19796,...",test
10604,270936,NaN,Maize,Telangana,Nirmal,Sarangapur,2023-02-12 0:00:00,2024-03-26 0:00:00,95,204,...,Maize,Drilling,Sprinkler,Canal,4,48,39,Rabi,"POLYGON ((78.25743 19.19557, 78.25743 19.19405...",test


In [19]:
# Function to scale pixel values to the 0–255 range for better visualization
def scale_band(band):
    """
    Scales pixel values of a single band to the 0-255 range.

    Parameters:
    - band: np.array, pixel values of the band

    Returns:
    - np.array, scaled pixel values in the 0-255 range
    """
    band = band.astype(np.float32)  # Ensure values are in float for scaling
    return 255 * (band - np.min(band)) / (np.max(band) - np.min(band))  # Scale to 0–255

# Function to visualize RGB image with scaling, contrast adjustments, and optional gamma correction
def visualize_rgb_image(file_path, gamma=0.6, contrast_stretch=True):
    """
    Visualize an RGB image using matplotlib with scaling, optional gamma correction, and contrast stretching.

    Parameters:
    - file_path: str, path to the RGB image file (GeoTIFF)
    - gamma: float, gamma correction factor (default=0.6)
    - contrast_stretch: bool, whether to apply contrast stretching (default=True)
    """
    # Open the image file
    with rasterio.open(file_path) as src:
        # Read RGB bands (assuming Sentinel-2 band order: Red=B4, Green=B3, Blue=B2)
        red = src.read(3)  # Band 4 for Red
        green = src.read(2)  # Band 3 for Green
        blue = src.read(1)  # Band 2 for Blue

        # Scale each band to the 0–255 range for better visualization
        red_scaled = scale_band(red)
        green_scaled = scale_band(green)
        blue_scaled = scale_band(blue)

        # Stack the scaled RGB bands into a single image
        rgb = np.dstack((red_scaled, green_scaled, blue_scaled)).astype(np.uint8)

        # Apply contrast stretching if specified
        if contrast_stretch:
            p2, p98 = np.percentile(rgb, (2, 98))  # Calculate 2nd and 98th percentiles for stretching
            rgb = exposure.rescale_intensity(rgb, in_range=(p2, p98))

        # Apply gamma correction to adjust brightness
        rgb = exposure.adjust_gamma(rgb, gamma=gamma)

        # Display the processed image using matplotlib
        plt.figure(figsize=(5, 5))
        plt.imshow(rgb)
        plt.axis('off')  # Hide axes for a cleaner look
        plt.title("RGB Composite (Red-Green-Blue) with Scaling, Contrast Stretch, and Gamma Correction")
        plt.show()

    return rgb

In [21]:
data

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,...,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,dataset
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25 0:00:00,2024-04-14 0:00:00,97,54,...,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,"POLYGON ((78.18079 17.97897, 78.1811 17.97948,...",train
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13 0:00:00,2024-04-26 0:00:00,82,58,...,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,"POLYGON ((78.17483 17.98151, 78.17649 17.98136...",train
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19 0:00:00,2024-04-28 0:00:00,92,91,...,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,"POLYGON ((78.16888 17.97673, 78.16968 17.97661...",train
3,1331836,Diseased,Paddy,Telangana,Medak,Kulcharam,2023-02-11 0:00:00,2024-11-04 0:00:00,91,52,...,Lentil,Transplanting,Flood,Canal,5,92,16,Rabi,"POLYGON ((78.16822 17.97469, 78.16955 17.97522...",train
4,1326579,Diseased,Paddy,Telangana,Medak,Kulcharam,2023-12-12 0:00:00,2024-05-19 0:00:00,94,55,...,Lentil,Transplanting,Flood,Canal,5,97,20,Rabi,"POLYGON ((78.17244 17.96962, 78.17303 17.96964...",train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10601,551119,NaN,Maize,Telangana,Nirmal,Sarangapur,2023-10-12 0:00:00,2024-07-03 0:00:00,92,225,...,Maize,Drilling,Drip,Groundwater,4,44,39,Rabi,"POLYGON ((78.259 19.22927, 78.25973 19.22915, ...",test
10602,916476,NaN,Maize,Telangana,Nirmal,Sarangapur,2023-05-11 0:00:00,2024-04-03 0:00:00,79,236,...,Maize,Seed Drilling,Drip,Canal,3,41,40,Rabi,"POLYGON ((78.25819 19.20201, 78.25792 19.20033...",test
10603,121860,NaN,Maize,Telangana,Nirmal,Sarangapur,2023-11-14 0:00:00,2024-03-04 0:00:00,87,210,...,Maize,Broadcasting,Sprinkler,Canal,3,43,41,Rabi,"POLYGON ((78.25546 19.19637, 78.2556 19.19796,...",test
10604,270936,NaN,Maize,Telangana,Nirmal,Sarangapur,2023-02-12 0:00:00,2024-03-26 0:00:00,95,204,...,Maize,Drilling,Sprinkler,Canal,4,48,39,Rabi,"POLYGON ((78.25743 19.19557, 78.25743 19.19405...",test


In [20]:
# Select a random image file from the dataset and visualize it
path = random.choice(data['tif_path'].values)
image = visualize_rgb_image(path)

KeyError: 'tif_path'

# Feature Engineering

**Utils**

In [31]:
# Example feature calculation functions
def calculate_ndvi(nir_band, red_band):
    """Calculate NDVI (Normalized Difference Vegetation Index)."""
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return np.nanmean(ndvi)

def calculate_evi(nir_band, red_band, blue_band):
    """Calculate EVI (Enhanced Vegetation Index)."""
    evi = 2.5 * (nir_band - red_band) / (nir_band + 6 * red_band - 7.5 * blue_band + 1)
    return np.nanmean(evi)

def calculate_ndwi(nir_band, green_band):
    """Calculate NDWI (Normalized Difference Water Index)."""
    ndwi = (green_band - nir_band) / (green_band + nir_band)
    return np.nanmean(ndwi)

def calculate_gndvi(nir_band, green_band):
    """Calculate GNDVI (Green Normalized Difference Vegetation Index)."""
    gndvi = (nir_band - green_band) / (nir_band + green_band)
    return np.nanmean(gndvi)

def calculate_savi(nir_band, red_band, L=0.5):
    """Calculate SAVI (Soil Adjusted Vegetation Index)."""
    savi = ((nir_band - red_band) / (nir_band + red_band + L)) * (1 + L)
    return np.nanmean(savi)

def calculate_msavi(nir_band, red_band):
    """Calculate MSAVI (Modified Soil Adjusted Vegetation Index)."""
    msavi = (2 * nir_band + 1 - np.sqrt((2 * nir_band + 1)**2 - 8 * (nir_band - red_band))) / 2
    return np.nanmean(msavi)

In [32]:
# Function to calculate features for each row in the data
def process_row_for_features(index, row):
    features = {'index': index}

    # Retrieve the TIFF file path and check if it's valid
    tif_path = row['tif_path']
    if not isinstance(tif_path, str):
        print(f"Skipping entry due to missing tif_path for index {index}")
        # Add NaN for all features if path is missing
        features.update({
            'ndvi': np.nan,
            'evi': np.nan,
            'ndwi': np.nan,
            'gndvi': np.nan,
            'savi': np.nan,
            'msavi': np.nan
        })
        return features

    # Open the TIFF file and read bands for feature calculation
    with rasterio.open(tif_path) as src:
        red = src.read(3)    # B4 for Red
        green = src.read(2)  # B3 for Green
        blue = src.read(1)   # B2 for Blue
        nir = src.read(4)    # B8 for NIR

        # Calculate each feature
        features['ndvi'] = calculate_ndvi(nir, red)
        features['evi'] = calculate_evi(nir, red, blue)
        features['ndwi'] = calculate_ndwi(nir, green)
        features['gndvi'] = calculate_gndvi(nir, green)
        features['savi'] = calculate_savi(nir, red)
        features['msavi'] = calculate_msavi(nir, red)

    return features

**Calculate Features**

In [33]:
# Use parallel processing to calculate features for each row
# `tqdm` provides a progress bar to track completion
new_features = Parallel(n_jobs=-1)(delayed(process_row_for_features)(index, row)
                                   for index, row in tqdm(data.iterrows(), total=len(data)))

  0%|          | 0/11791 [00:00<?, ?it/s]

KeyError: 'tif_path'

**Merge New Features with Original Data**

In [34]:
# Convert list of features to a DataFrame and merge with the original data
new_features_df = pd.DataFrame(new_features).set_index('index')
data = data.join(new_features_df)

NameError: name 'new_features' is not defined

In [35]:
data.head()

,type,features
0,FeatureCollection,"{'id': '0', 'type': 'Feature', 'properties': {..."
1,FeatureCollection,"{'id': '1', 'type': 'Feature', 'properties': {..."
2,FeatureCollection,"{'id': '2', 'type': 'Feature', 'properties': {..."
3,FeatureCollection,"{'id': '3', 'type': 'Feature', 'properties': {..."
4,FeatureCollection,"{'id': '4', 'type': 'Feature', 'properties': {..."


**Split Dataset into Train and Test**

In [36]:
# Split the data into train and test sets based on the 'dataset' column
train = data[data.dataset == 'train'].reset_index(drop=True)
test = data[data.dataset == 'test'].reset_index(drop=True)

AttributeError: 'DataFrame' object has no attribute 'dataset'

In [28]:
# Define columns to drop before encoding and training
to_drop = [
    'geometry', 'tif_path',  # Spatial and file path information
    'dataset', 'FarmID',     # Columns that are not needed for modeling
]

# Drop the specified columns from train and test sets
train.drop(columns=to_drop, inplace=True)
test.drop(columns=to_drop, inplace=True)


In [29]:
# Concatenate train and test datasets for uniform encoding of categorical columns
data_combined = pd.concat([train, test], ignore_index=True)

In [30]:
# Create a mapping for the target variable 'category' to numeric labels, excluding NaN values
category_mapper = {label: idx for idx, label in enumerate(data_combined['category'].unique()) if pd.notna(label)}
idx_to_category_mapper = {idx: label for idx, label in enumerate(data_combined['category'].unique()) if pd.notna(label)}

# Map the 'category' column to the new 'target' column using the mapping, handling NaN values as needed
data_combined['target'] = data_combined['category'].map(category_mapper)

# Drop the original 'category' column now that we have the 'target' column
data_combined.drop(columns=['category'], inplace=True)

In [31]:
# Identify categorical columns to encode
categorical_cols = data_combined.select_dtypes(include=['object']).columns

# Apply Label Encoding for all categorical columns
for col in categorical_cols:
    encoder = LabelEncoder()
    data_combined[col] = encoder.fit_transform(data_combined[col])

In [32]:
# Split the combined encoded data back into train and test
train_encoded = data_combined[data_combined.index < len(train)].reset_index(drop=True)
test_encoded = data_combined[data_combined.index >= len(train)].reset_index(drop=True)

# Model Training

In [33]:
# Separate features and target variable in the train set
X = train_encoded.drop(columns=['target'])
y = train_encoded['target'].astype(int)

In [34]:
# Stratified split based on 'target' for training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [35]:
# Configuration dictionary for LightGBM parameters
config_dict = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': len(category_mapper),  # Number of classes in the target
    'num_leaves': 81,
    'learning_rate': 0.01,
    'n_estimators': 500,
    'random_state': 42,
    'force_row_wise': 'true'
}

# Initialize LightGBM classifier with config_dict parameters
model = LGBMClassifier(**config_dict)

# Fit the model with an evaluation set for early stopping and progress tracking
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
)

[LightGBM] [Info] Total Bins 2562
[LightGBM] [Info] Number of data points in the train set: 7020, number of used features: 22
[LightGBM] [Info] Start training from score -0.195918
[LightGBM] [Info] Start training from score -2.792733
[LightGBM] [Info] Start training from score -2.795062
[LightGBM] [Info] Start training from score -2.890372


LGBMClassifier(force_row_wise='true', learning_rate=0.01,
               metric='multi_logloss', n_estimators=500, num_class=4,
               num_leaves=81, objective='multiclass', random_state=42)

In [36]:
# Predict on the validation set
y_pred = model.predict(X_valid)

# Generate classification report and weighted F1 score
classification_rep = classification_report(y_valid, y_pred, target_names=list(category_mapper.keys()))
f1 = f1_score(y_valid, y_pred, average='weighted')

# Print model evaluation metrics
print("Classification Report:\n", classification_rep)
print("\nWeighted F1 Score:", f1)

Classification Report:
               precision    recall  f1-score   support

     Healthy       0.82      1.00      0.90      1443
    Diseased       0.50      0.01      0.02       107
       Pests       0.00      0.00      0.00       107
    Stressed       0.00      0.00      0.00        98

    accuracy                           0.82      1755
   macro avg       0.33      0.25      0.23      1755
weighted avg       0.71      0.82      0.74      1755


Weighted F1 Score: 0.743073912831982


 # Submit

In [37]:
# Load submission file
sample_submission = pd.read_csv(root_path / "SampleSubmission.csv")

# Predict on the test set
test_predictions = model.predict(test_encoded.drop(columns=['target']))  # Drop 'FarmID' if it exists in test_encoded

# Fill sample_submission.csv & Save
sample_submission['Target'] = test_predictions
sample_submission['Target'] = sample_submission['Target'].astype(int).map(idx_to_category_mapper)
sample_submission.to_csv('submission.csv', index=False)

In [40]:
sample_submission.head()

,ID,Target
0,85197,Healthy
1,779677,Healthy
2,1331840,Healthy
3,483053,Healthy
4,542945,Healthy
